Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/azure-integration/remote-explanation/explain-model-on-amlcompute.png)

# Train and explain models remotely via Azure Machine Learning Compute


_**This notebook showcases how to use the Azure Machine Learning Interpretability SDK to train and explain a regression model remotely on an Azure Machine Leanrning Compute Target (AMLCompute).**_




## Table of Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
    1. Initialize a Workspace
    1. Create an Experiment
    1. Introduction to AmlCompute
    1. Submit an AmlCompute run in a few different ways
        1. Option 1: Provision as a run based compute target 
        1. Option 2: Provision as a persistent compute target (Basic)
        1. Option 3: Provision as a persistent compute target (Advanced)
1. Additional operations to perform on AmlCompute
1. [Download model explanations from Azure Machine Learning Run History](#Download)
1. [Visualize explanations](#Visualize)
1. [Next steps](#Next)

## Introduction

This notebook showcases how to train and explain a regression model remotely via Azure Machine Learning Compute (AMLCompute), and download the calculated explanations locally for visualization.
It demonstrates the API calls that you need to make to submit a run for training and explaining a model to AMLCompute, download the compute explanations remotely, and visualizing the global and local explanations via a visualization dashboard that provides an interactive way of discovering patterns in model predictions and downloaded explanations.

We will showcase one of the tabular data explainers: TabularExplainer (SHAP).

Problem: Boston Housing Price Prediction with scikit-learn (train a model and run an explainer remotely via AMLCompute, and download and visualize the remotely-calculated explanations.)

![explanations-run-history](./img/explanations-run-history.png)

## Setup

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.69


## Initialize a Workspace

Connect to the workspace

In [2]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

ws-ahcaneuap05
rg-ahcan
eastus2euap
13e50845-67bc-4ac5-94db-48d493a6d9e8


In [3]:
# if you did not create the dataset in the first part of the workshop, you can just create it now
web_path ='https://raw.githubusercontent.com/danielsc/azureml-workshop-2019/master/data/IBM-Employee-Attrition.csv'
attritionDataSet = Dataset.Tabular.from_delimited_files(path=web_path)
attritionDataSet.register(ws, name='IBM-Employee-Attrition')
attritionData = attritionDataSet.to_pandas_dataframe()
attritionData.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [4]:
from azureml.core import Experiment
experiment_name = 'explainer-remote-run-on-amlcompute'
experiment = Experiment(workspace=ws, name=experiment_name)

## Introduction to AmlCompute

Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.



## Submit an AmlCompute run in a few different ways

First lets check which VM families are available in your region. Azure is a regional service and some specialized SKUs (especially GPUs) are only available in certain regions. Since AmlCompute is created in the region of your workspace, we will use the supported_vms () function to see if the VM family we want to use ('STANDARD_D2_V2') is supported.

You can also pass a different region to check availability and then re-create your workspace in that region through the [configuration notebook](../../../configuration.ipynb)

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [5]:
import os
import shutil

project_folder = './explainer-remote-run-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train_explain.py', project_folder)

'./explainer-remote-run-on-amlcompute/train_explain.py'

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

### Fetch or create the compute target 

We are going to use the compute target you had created before (make sure you provide the same name here in the variable `cpu_cluster_name`. 

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu"

cpu_cluster = ws.compute_targets[cpu_cluster_name]

The above code will throw an error if you didn't set the cluster up in the first part of the workshop. In that case, you can provision a cluster here by executing the next cell.

Note: By default a cluster autoscales from 0 nodes and provisions dedicated VMs to run your job in a container. This is useful when you want to continously re-use the same target, debug it between jobs or simply share the resource with other users of your workspace.

* `vm_size`: VM family of the nodes provisioned by AmlCompute. Simply choose from the supported_vmsizes() above
* `max_nodes`: Maximum nodes to autoscale to while running a job on AmlCompute

In [7]:
## if you didn't create the cluster in the first part of the workshop, this code will create it
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                       max_nodes=10, 
                                                       idle_seconds_before_scaledown=7200)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


### Configure & Run

In [9]:
from azureml.train.estimator import Estimator

pip_packages = [
    'azureml-defaults', 'azureml-contrib-interpret', 'azureml-core', 'azureml-telemetry',
    'azureml-interpret', 'sklearn-pandas', 'azureml-dataprep', 'joblib'
]

estimator = Estimator(source_directory=project_folder, 
                      compute_target=cpu_cluster,
                      entry_script='train_explain.py',
                      pip_packages=pip_packages,
                      conda_packages=['scikit-learn'],
                      inputs=[ws.datasets['IBM-Employee-Attrition'].as_named_input('attrition')])

run = experiment.submit(estimator)
run

Experiment,Id,Type,Status,Details Page,Docs Page
explainer-remote-run-on-amlcompute,explainer-remote-run-on-amlcompute_1571356196_31fe70f3,azureml.scriptrun,Starting,Link to Azure Portal,Link to Documentation


In [10]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Additional operations to perform on AmlCompute

You can perform more operations on AmlCompute such as updating the node counts or deleting the compute. 

In [11]:
# Get_status () gets the latest status of the AmlCompute target
cpu_cluster.get_status().serialize()


{'currentNodeCount': 1,
 'targetNodeCount': 1,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'idleNodeCount': 1,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2019-10-17T21:50:40.172000+00:00',
 'errors': None,
 'creationTime': '2019-10-17T21:39:20.835643+00:00',
 'modifiedTime': '2019-10-17T23:43:38.454271+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 0,
  'maxNodeCount': 10,
  'nodeIdleTimeBeforeScaleDown': 'PT7200S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_D2_V2'}

In [12]:
# Update () takes in the min_nodes, max_nodes and idle_seconds_before_scaledown and updates the AmlCompute target
# cpu_cluster.update(min_nodes=1)
# cpu_cluster.update(max_nodes=10)
cpu_cluster.update(idle_seconds_before_scaledown=7200)
# cpu_cluster.update(min_nodes=2, max_nodes=4, idle_seconds_before_scaledown=600)

In [13]:
# Delete () is used to deprovision and delete the AmlCompute target. Useful if you want to re-use the compute name 
# 'cpu-cluster' in this case but use a different VM family for instance.

# cpu_cluster.delete()

In [14]:
run.wait_for_completion(show_output=True)

RunId: explainer-remote-run-on-amlcompute_1571356196_31fe70f3
Web View: https://mlworkspace.azure.ai/portal/subscriptions/13e50845-67bc-4ac5-94db-48d493a6d9e8/resourceGroups/rg-ahcan/providers/Microsoft.MachineLearningServices/workspaces/ws-ahcaneuap05/experiments/explainer-remote-run-on-amlcompute/runs/explainer-remote-run-on-amlcompute_1571356196_31fe70f3

Streaming azureml-logs/55_azureml-execution-tvmps_a31138435db3f566d455bd4f927c61518cd6d95d480d3245601df5b07c24c8d7_d.txt

2019-10-17T23:50:06Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_35f6d26d86642ee60b4e82dd0f80e99b
Digest: sha256:ff0ff342f108dfc814b5c02324302a0fe7f8b85afab83e74cda0f9125934bd50
Status: Image is up to date for wsahcaneuap03e8d12c4.azurecr.io/azureml/azureml_35f6d26d86642ee60b4e82dd0f80e99b:latest
17495dc1e59ed60f63166f4edf36925e3643943111c9474e8624b1960e659e77
2019/10/17 23:50:08 Version: 3.0.00998.0001 Branch: master Commit: 20ee96da
2019/10/17 23:50

{'runId': 'explainer-remote-run-on-amlcompute_1571356196_31fe70f3',
 'target': 'cpu',
 'status': 'Completed',
 'startTimeUtc': '2019-10-17T23:50:09.333934Z',
 'endTimeUtc': '2019-10-17T23:50:55.082831Z',
 'properties': {'_azureml.ComputeTargetType': 'batchai',
  'ContentSnapshotId': 'd0e31b73-9d90-4de6-bbbf-7d4ae924a601',
  'azureml.git.repository_uri': 'https://github.com/danielsc/azureml-workshop-2019',
  'mlflow.source.git.repoURL': 'https://github.com/danielsc/azureml-workshop-2019',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'af8aacc7dc810de10430b9f6c84076b6c4545b54',
  'mlflow.source.git.commit': 'af8aacc7dc810de10430b9f6c84076b6c4545b54',
  'azureml.git.dirty': 'True',
  'AzureML.DerivedImageName': 'azureml/azureml_35f6d26d86642ee60b4e82dd0f80e99b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'model_type': 'classification',
  'explainer': 'tabular'},
 'i

## Download 
### 1. Download model explanation data.

In [15]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(run)
global_explanation = client.download_model_explanation()


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [16]:
# Get the top k (e.g., 4) most important features with their importance values
global_explanation_topk = client.download_model_explanation(top_k=4)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [17]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

global importance values: [0.7957616026886991, 0.5064862653239521, 0.42413509706782865, 0.4125222270210433]
global importance names: ['OverTime', 'NumCompaniesWorked', 'JobSatisfaction', 'TotalWorkingYears']


### 2. Download model and test set files

In [18]:
# retrieve model for visualization and deployment
from azureml.core.model import Model
import joblib

run.download_file('original_model.pkl')
original_model = joblib.load('original_model.pkl')

run.download_file('x_test_ibm.pkl')
x_test = joblib.load('x_test_ibm.pkl')

## Visualize
Load the visualization dashboard

In [19]:
from azureml.contrib.interpret.visualize import ExplanationDashboard

In [20]:
ExplanationDashboard(global_explanation, original_model, x_test)

ExplanationWidget(value={'predictedY': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0…